In [ ]:
from datetime import datetime
import requests
import json
import pandas as pd
import plotly
import plotly.express as px
import os
from PIL import Image
import geopy
from geopy.geocoders import GoogleV3
import plotly.graph_objects as go
# To be added as an environment variable: 
from config import key, GOOGLE_MAP_KEY

In [ ]:
# start and end date function

# wrap with a docstring and name 

# This function grabs today's date and converts it into the appropriate format for use in an API request for historic weather.
# This function could be in a utils folder because it doesn't need anything from the user.
# Note that the from datetime import datetime would also need to be in the utils folder.

def start_end(year):
    today = datetime.now()
    if len(str(today.month)) < 2:
        month = '0' + str(today.month)
    else:
        month = str(today.month)
    if len(str(today.day)) < 2:
        day = '0' + str(today.day)
    else:
        day = str(today.day)
    start_date = year + '-' + month + '-' + day
    end_date = start_date
    return start_date, end_date


In [ ]:
# This could be a radio button, range slider, dropdown, etc.

historical_year = input("What year do you select? ")

In [ ]:
# Passing the start and end dates through the start_end() funcion.
# Setting them to the same allows for capturing a single day in the API request
# This could be in the utils folder and the 2 variables imported into the app.
start_date, end_date = start_end(historical_year)


In [ ]:
# Verifying appropriate format for API request
# Unneccessary for the dash app.
start_date

In [ ]:
# Capturing user's address for geocoding
# This would be converted to a dcc.Input or mantine equivalent.
# This could be an option if the user does not allow for geolocation.
# Also consider a dropdown or radio that allows the user to choose between an address and geolocation.
address = input("What is your address? ") 

In [ ]:
# Establishing the geocoder
# This could be done in a utils folder
# Note that the necessary dependencies would also need to be included.
# Source: https://towardsdatascience.com/geocode-with-python-161ec1e62b89
# Source: https://geopy.readthedocs.io/en/stable/#module-geopy.geocoders
locator = GoogleV3(api_key=GOOGLE_MAP_KEY, user_agent="newGeocoder")

In [ ]:
# Geocoding the user's address
# This would need to happen in the body of the app
location = locator.geocode(address)

In [ ]:
# setting lat and lon for address
# This would need to happen in the body of the app
lat = location.latitude
lon = location.longitude

In [ ]:
# This would all potentially need to take place in a callback associated with the address Input.
# Think here: Could this be functionalized? Does it make sense to functionalize it?
# Making the API requuest to open-meto
hist_weather = requests.get(f"https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={start_date}&end_date={end_date}&hourly=temperature_2m&daily=temperature_2m_max,temperature_2m_min&timezone=auto&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch").json()

# Capturing the temperature:
temp = hist_weather['hourly']['temperature_2m']
# Capturing the time:
time = hist_weather['hourly']['time']
# Creating a dictionary to prep for a pandas dataframe:
data = {'Hour': time, 'Temperature': temp}
# Converting dictionary to a Pandas dataframe:
hw_df = pd.DataFrame(data=data)
    
    

In [ ]:
# Creating a line graph of the day's temperature.
fig = px.line(hw_df, x='Hour', y='Temperature')
fig

In [ ]:
print(lat,lon)

In [ ]:
print(hw_df.min())
print(hw_df.max())


In [ ]:
hw_df

In [ ]:
# This data cleaning should be functionalized.
# Would then call the function from within a callback function.
clean_hw = hw_df['Hour'].str.split("T", expand=True)

clean_hw["Temperature"] = hw_df['Temperature']
clean_hw.rename(columns={0:"Date", 1:"Hour"}, inplace=True)

In [ ]:
fig_1 = px.line(clean_hw, x='Hour', y='Temperature')
fig_1

In [ ]:
current_weather = requests.get(f"http://api.weatherapi.com/v1/forecast.json?key={key}&q={lat},{lon}&days=1&aqi=no&alerts=no").json()


In [ ]:
current_weather

In [ ]:
print(type(current_weather))

In [ ]:
# Extracting hour and temperature data from current forecast and storing in a dictionary
# Converting that dictionary to a pandas DataFrame

current_data = {"Hour": [x['time'] for x in current_weather['forecast']['forecastday'][0]['hour']], "Temperature": [y['temp_f'] for y in current_weather['forecast']['forecastday'][0]['hour']]}
current_w = pd.DataFrame(current_data)
current_w



In [ ]:
# This data cleaning should be functionalized
# Then call the function within a callback function.
# Think this could be just one function that has the split as an argument.

clean_current = current_w['Hour'].str.split(expand=True)
clean_current['Temperature'] = current_w['Temperature']
clean_current.rename(columns={0:"Date", 1:"Hour"}, inplace=True)
clean_current

In [ ]:
# Considering a data cleaning function:
# This function would need to be added the utlils folder

def weather_clean(df, splitter):
    df1 = df['Hour'].str.split(splitter, expand=True)
    df1['Temperature'] = df['Temperature']
    df1.rename(columns={0:"Date", 1:"Hour"}, inplace=True)
    return df1

In [ ]:
clean_historical_1 = weather_clean(hw_df, "T")

clean_historical_1

In [ ]:
current_clean_2 = weather_clean(current_w, " ")
current_clean_2

In [ ]:
# Problem: need to get the 2 traces to have the correct hover data
# Consider: Changing the data so that only the hour is present in the x-axis (year present in hover)
fig_1.add_trace(
    go.Scatter(
        x=clean_current['Hour'],
        y=clean_current['Temperature'],
        mode="lines",
        line=go.scatter.Line(color="gray"),
        showlegend=True))

